# LlamaIndex Workflows

Workflows provide structured control flow for complex AI applications. They allow you to define multi-step processes with branching, looping, and error handling.

## Learning Objectives

By the end of this notebook, you will:
1. Understand the Workflow architecture
2. Build sequential workflows
3. Implement branching and conditional logic
4. Create parallel execution patterns
5. Handle events and state management

---

## What are Workflows?

Workflows are **event-driven** pipelines that:
- Define explicit control flow
- Handle complex multi-step processes
- Support parallel execution
- Enable state management between steps

### Workflow vs Agent

| Aspect | Agent | Workflow |
|--------|-------|----------|
| Control | LLM decides next step | Explicit code defines flow |
| Predictability | Variable | Deterministic |
| Debugging | Harder | Easier |
| Best for | Open-ended tasks | Structured processes |

In [ ]:
# Setup
import nest_asyncio
nest_asyncio.apply()

import asyncio
from dotenv import load_dotenv
load_dotenv()

from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    Settings,
)
from llama_index.core.workflow import (
    Workflow,
    StartEvent,
    StopEvent,
    Event,
    step,
)
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

# Configure
Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0)
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

print("✓ Setup complete!")

## 1. Basic Workflow Structure

Workflows consist of:
- **Events**: Messages that flow between steps
- **Steps**: Functions that process events
- **StartEvent**: Triggers the workflow
- **StopEvent**: Ends the workflow with a result

In [ ]:
# Define a simple workflow
class SimpleWorkflow(Workflow):
    """A basic workflow that processes text."""
    
    @step
    async def process_input(self, ev: StartEvent) -> StopEvent:
        """Process the input and return result."""
        input_text = ev.input
        
        # Simple processing
        processed = input_text.upper()
        word_count = len(input_text.split())
        
        result = f"Processed: {processed}\nWord count: {word_count}"
        
        return StopEvent(result=result)

print("✓ SimpleWorkflow defined!")

In [ ]:
# Run the workflow
async def run_simple_workflow():
    workflow = SimpleWorkflow()
    result = await workflow.run(input="Hello world from LlamaIndex workflows!")
    return result

result = asyncio.run(run_simple_workflow())
print("Result:")
print(result)

## 2. Multi-Step Workflow with Custom Events

Define custom events to pass data between steps:

In [ ]:
from pydantic import BaseModel
from typing import Optional

# Define custom events
class ValidationEvent(Event):
    """Event after input validation."""
    text: str
    is_valid: bool

class ProcessedEvent(Event):
    """Event after text processing."""
    original: str
    processed: str
    word_count: int

class EnrichedEvent(Event):
    """Event after enrichment with LLM."""
    summary: str
    metadata: dict

print("✓ Custom events defined!")

In [ ]:
class TextProcessingWorkflow(Workflow):
    """A multi-step workflow for text processing."""
    
    @step
    async def validate_input(self, ev: StartEvent) -> ValidationEvent:
        """Step 1: Validate the input text."""
        print("Step 1: Validating input...")
        
        text = ev.input
        is_valid = len(text) > 0 and len(text) < 10000
        
        return ValidationEvent(text=text, is_valid=is_valid)
    
    @step
    async def process_text(self, ev: ValidationEvent) -> ProcessedEvent | StopEvent:
        """Step 2: Process the text if valid."""
        print("Step 2: Processing text...")
        
        if not ev.is_valid:
            return StopEvent(result="Error: Invalid input")
        
        processed = ev.text.strip()
        word_count = len(processed.split())
        
        return ProcessedEvent(
            original=ev.text,
            processed=processed,
            word_count=word_count,
        )
    
    @step
    async def enrich_with_llm(self, ev: ProcessedEvent) -> EnrichedEvent:
        """Step 3: Enrich with LLM analysis."""
        print("Step 3: Enriching with LLM...")
        
        # Use LLM to generate summary
        llm = Settings.llm
        response = await llm.acomplete(
            f"Summarize this text in one sentence: {ev.processed}"
        )
        
        return EnrichedEvent(
            summary=str(response),
            metadata={
                "word_count": ev.word_count,
                "char_count": len(ev.processed),
            }
        )
    
    @step
    async def format_output(self, ev: EnrichedEvent) -> StopEvent:
        """Step 4: Format final output."""
        print("Step 4: Formatting output...")
        
        result = f"""
=== Processing Complete ===
Summary: {ev.summary}
Metadata: {ev.metadata}
"""
        return StopEvent(result=result)

print("✓ TextProcessingWorkflow defined!")

In [ ]:
# Run the multi-step workflow
async def run_text_workflow():
    workflow = TextProcessingWorkflow(timeout=60)
    
    input_text = """
    Artificial intelligence is transforming how we work and live. 
    Machine learning algorithms can now recognize patterns in data 
    that humans might miss. This technology is being applied in 
    healthcare, finance, and many other industries.
    """
    
    result = await workflow.run(input=input_text)
    return result

print("Running multi-step workflow...\n")
result = asyncio.run(run_text_workflow())
print(result)

## 3. Workflow with RAG

Integrate RAG into a workflow:

In [ ]:
# Load documents and create index
documents = SimpleDirectoryReader("../data/sample_docs").load_data()
index = VectorStoreIndex.from_documents(documents, show_progress=True)
query_engine = index.as_query_engine(similarity_top_k=3)

print("\n✓ Index ready!")

In [ ]:
# Define RAG workflow events
class QueryEvent(Event):
    query: str

class RetrievalEvent(Event):
    query: str
    contexts: list
    
class AnswerEvent(Event):
    answer: str
    sources: list

class RAGWorkflow(Workflow):
    """A RAG workflow with explicit steps."""
    
    def __init__(self, query_engine, **kwargs):
        super().__init__(**kwargs)
        self.query_engine = query_engine
    
    @step
    async def parse_query(self, ev: StartEvent) -> QueryEvent:
        """Parse and validate the query."""
        print("Step 1: Parsing query...")
        query = ev.input.strip()
        return QueryEvent(query=query)
    
    @step
    async def retrieve(self, ev: QueryEvent) -> RetrievalEvent:
        """Retrieve relevant documents."""
        print("Step 2: Retrieving documents...")
        
        # Get retriever from query engine
        retriever = self.query_engine._retriever
        nodes = retriever.retrieve(ev.query)
        
        contexts = [node.text for node in nodes]
        
        print(f"  Retrieved {len(contexts)} contexts")
        return RetrievalEvent(query=ev.query, contexts=contexts)
    
    @step
    async def generate(self, ev: RetrievalEvent) -> AnswerEvent:
        """Generate answer using LLM."""
        print("Step 3: Generating answer...")
        
        # Build prompt with context
        context_str = "\n\n".join(ev.contexts)
        prompt = f"""Based on the following context, answer the question.

Context:
{context_str}

Question: {ev.query}

Answer:"""
        
        llm = Settings.llm
        response = await llm.acomplete(prompt)
        
        return AnswerEvent(
            answer=str(response),
            sources=ev.contexts[:2],  # Include top 2 sources
        )
    
    @step
    async def format_response(self, ev: AnswerEvent) -> StopEvent:
        """Format the final response."""
        print("Step 4: Formatting response...")
        
        result = {
            "answer": ev.answer,
            "sources": [s[:100] + "..." for s in ev.sources],
        }
        
        return StopEvent(result=result)

print("✓ RAGWorkflow defined!")

In [ ]:
# Run RAG workflow
async def run_rag_workflow():
    workflow = RAGWorkflow(query_engine=query_engine, timeout=60)
    result = await workflow.run(input="What is machine learning?")
    return result

print("Running RAG workflow...\n")
result = asyncio.run(run_rag_workflow())

print("\n=== Result ===")
print(f"Answer: {result['answer']}")
print(f"\nSources used: {len(result['sources'])}")

## 4. Branching Workflow

Workflows can have conditional branching based on events:

In [ ]:
# Define events for branching
class ClassifyEvent(Event):
    query: str
    query_type: str  # "factual", "analytical", "creative"

class FactualEvent(Event):
    query: str

class AnalyticalEvent(Event):
    query: str

class CreativeEvent(Event):
    query: str

class BranchingWorkflow(Workflow):
    """Workflow with conditional branching based on query type."""
    
    @step
    async def classify_query(self, ev: StartEvent) -> ClassifyEvent:
        """Classify the query type using LLM."""
        print("Classifying query...")
        
        llm = Settings.llm
        prompt = f"""Classify this query into one of: factual, analytical, creative.
Just respond with one word.

Query: {ev.input}

Classification:"""
        
        response = await llm.acomplete(prompt)
        query_type = str(response).strip().lower()
        
        # Normalize
        if "fact" in query_type:
            query_type = "factual"
        elif "analy" in query_type:
            query_type = "analytical"
        else:
            query_type = "creative"
        
        print(f"  Query type: {query_type}")
        return ClassifyEvent(query=ev.input, query_type=query_type)
    
    @step
    async def route_query(self, ev: ClassifyEvent) -> FactualEvent | AnalyticalEvent | CreativeEvent:
        """Route to appropriate handler."""
        print(f"Routing to {ev.query_type} handler...")
        
        if ev.query_type == "factual":
            return FactualEvent(query=ev.query)
        elif ev.query_type == "analytical":
            return AnalyticalEvent(query=ev.query)
        else:
            return CreativeEvent(query=ev.query)
    
    @step
    async def handle_factual(self, ev: FactualEvent) -> StopEvent:
        """Handle factual queries - focus on accuracy."""
        print("Handling as FACTUAL query...")
        
        llm = Settings.llm
        response = await llm.acomplete(
            f"Provide a concise, factual answer: {ev.query}"
        )
        
        return StopEvent(result={
            "type": "factual",
            "answer": str(response),
        })
    
    @step
    async def handle_analytical(self, ev: AnalyticalEvent) -> StopEvent:
        """Handle analytical queries - provide analysis."""
        print("Handling as ANALYTICAL query...")
        
        llm = Settings.llm
        response = await llm.acomplete(
            f"Provide a detailed analysis with multiple perspectives: {ev.query}"
        )
        
        return StopEvent(result={
            "type": "analytical",
            "answer": str(response),
        })
    
    @step
    async def handle_creative(self, ev: CreativeEvent) -> StopEvent:
        """Handle creative queries - be imaginative."""
        print("Handling as CREATIVE query...")
        
        llm = Settings.llm
        response = await llm.acomplete(
            f"Provide a creative and engaging response: {ev.query}"
        )
        
        return StopEvent(result={
            "type": "creative",
            "answer": str(response),
        })

print("✓ BranchingWorkflow defined!")

In [ ]:
# Test branching workflow with different query types
async def test_branching():
    workflow = BranchingWorkflow(timeout=60)
    
    queries = [
        "What is the capital of France?",  # Factual
        "Why do some companies succeed while others fail?",  # Analytical
        "Write a haiku about programming.",  # Creative
    ]
    
    for query in queries:
        print("\n" + "="*60)
        print(f"Query: {query}")
        print("="*60)
        
        result = await workflow.run(input=query)
        print(f"\nType: {result['type']}")
        print(f"Answer: {result['answer'][:200]}...")

asyncio.run(test_branching())

## 5. Error Handling in Workflows

Handle errors gracefully within workflows:

In [ ]:
class ErrorEvent(Event):
    error_message: str
    step_name: str

class RobustWorkflow(Workflow):
    """Workflow with error handling."""
    
    @step
    async def risky_step(self, ev: StartEvent) -> ProcessedEvent | ErrorEvent:
        """A step that might fail."""
        try:
            value = int(ev.input)
            if value < 0:
                raise ValueError("Negative values not allowed")
            
            return ProcessedEvent(
                original=ev.input,
                processed=str(value * 2),
                word_count=1,
            )
        except (ValueError, TypeError) as e:
            return ErrorEvent(
                error_message=str(e),
                step_name="risky_step",
            )
    
    @step
    async def handle_success(self, ev: ProcessedEvent) -> StopEvent:
        """Handle successful processing."""
        return StopEvent(result={
            "status": "success",
            "result": ev.processed,
        })
    
    @step
    async def handle_error(self, ev: ErrorEvent) -> StopEvent:
        """Handle errors gracefully."""
        return StopEvent(result={
            "status": "error",
            "error": ev.error_message,
            "step": ev.step_name,
        })

print("✓ RobustWorkflow defined!")

In [ ]:
# Test error handling
async def test_error_handling():
    workflow = RobustWorkflow(timeout=30)
    
    test_inputs = ["42", "-5", "not a number"]
    
    for inp in test_inputs:
        print(f"\nInput: '{inp}'")
        result = await workflow.run(input=inp)
        print(f"Result: {result}")

asyncio.run(test_error_handling())

## 6. Summary

You've learned how to build workflows with LlamaIndex:

### Key Takeaways

| Concept | Description |
|---------|-------------|
| **Events** | Messages between workflow steps |
| **Steps** | Async functions that process events |
| **Branching** | Conditional routing based on data |
| **Error Handling** | Graceful failure recovery |

### When to Use Workflows

- **Use Workflows** when you need predictable, explicit control flow
- **Use Agents** when you want LLM-driven decision making
- **Combine both** for complex applications

### Next Steps

In the next notebook, we'll explore creating custom components.

---

## Exercises

1. **Document processing**: Create a workflow that processes documents through multiple stages

2. **Retry logic**: Add retry capability for failed steps

3. **Parallel steps**: Implement steps that run in parallel

4. **State management**: Add global state that persists across steps

In [ ]:
# Exercise space
# Build your own workflow here!